In [1]:
'''
author: Felix Hol
date: 2020 Nov 25
content: code to calculate distance of nearest mosquito and identify its particle number
'''

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import itertools as it
import pandas as pd
import pims
import skimage
from skimage import data, io, util
from skimage.measure import label, regionprops
from skimage.morphology import binary_dilation, erosion, dilation, opening, binary_closing, closing, white_tophat, remove_small_objects, disk, black_tophat, skeletonize, convex_hull_image
import scipy
import trackpy as tp
import pylab
import math
from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime
from tqdm.notebook import tnrange, tqdm
import pickle
import glob
import cv2 as cv

In [2]:
#### set directories where to get images and where to store output, and specifics of experiment
dataDir = '/home/felix/biteData/invitation//'
saveDir = '/home/felix/biteData/invitation/'

In [3]:
experiments = glob.glob(saveDir + 'aeg_ctrl*1105_tracks.pkl')

In [4]:
experiments

['/home/felix/biteData/invitation/aeg_ctrl1_2_1105_tracks.pkl',
 '/home/felix/biteData/invitation/aeg_ctrl3_4_1105_tracks.pkl']

In [5]:
'''
create a new dataframe based on tracking dataframe having nearest neighbor distances for all particles.
keepN truncates tracks longer than N points
'''

areaWidth = 2806
areaHigth = 2736
keepN = 50
Area = areaWidth * areaHigth ### in pix, check what actualy numbers should be (e.g. ~1893 * 1911 for invitation)
perimeter = 2 * areaHigth + 2 * areaWidth ### in pix, check what actualy numbers should be (~7608 for invitation)



for exp in tqdm(experiments):
    T = pd.read_pickle(exp)
    for p in T.particle.unique():
        if len(T[(T.particle == p)]) > keepN + 1:
            start = np.min(T.loc[T.particle == p].frame)
            T = T.drop(T[(T.particle == p) & (T.frame > start + keepN)].index)
    T['nnDist'] = np.nan
    T['nnP'] = np.nan

    for f in tqdm(T.frame.unique()):
        t = T.loc[T.frame == f]
        if len(t) > 1:
            for p in t.particle.unique():
                xp = t.loc[t.particle == p].x.values
                yp = t.loc[t.particle == p].y.values
                dist = []
                others = np.delete(t.particle.unique(), np.where(t.particle.unique() == p))
                for o in others:
                    xo = t.loc[t.particle == o].x.values
                    yo = t.loc[t.particle == o].y.values
                    D = np.sqrt(np.abs(xp - xo)**2 + np.abs(yp - yo)**2)
                    dist = np.append(dist, D)
                T.loc[T.loc[(T.frame == f) & (T.particle == p)].index, 'nnDist'] = np.min(dist)
                T.loc[T.loc[(T.frame == f) & (T.particle == p)].index, 'nnP'] = int(others[np.argmin(dist)])
    T.to_pickle(saveDir + os.path.splitext(os.path.basename(exp))[0] + 'NN_keep_'+ str(keepN) + '.pkl')
    P = T.copy()
    P = P[~P.nnDist.isna()]
    
    NN = pd.DataFrame(columns=['frame', 'N', 'nnDo', 'nnDe'])
    for f in tqdm(P.frame.unique()):
        pP = P.loc[P.frame == f]
        N = len(pP)
        nnDo = np.mean(pP.nnDist)
        nnDe = 1/2 * np.sqrt(Area / N) + (0.0514 * perimeter) / N + (0.041 * perimeter) / (N * np.sqrt(N))
        NN = NN.append({'frame': f, 'N': N, 'nnDo': nnDo, 'nnDe': nnDe}, ignore_index = True)

    NN['nnI'] = NN.nnDo / NN.nnDe  
    NN['Z'] = (NN.nnDo - NN.nnDe) / (0.26136 / (np.sqrt((NN.N ** 2) / Area)))
    NN['Zabs'] = np.abs(NN.Z)
    NN.to_pickle(saveDir + os.path.splitext(os.path.basename(exp))[0] + 'NNframe_keep_'+ str(keepN) + '.pkl')

225294
/home/felix/biteData/invitation/aeg_ctrl1_2_1105_tracks.pkl
29140



29140
18980



150177
/home/felix/biteData/invitation/aeg_ctrl3_4_1105_tracks.pkl
14975



14975
9059
